In [ ]:
%load_ext autoreload
%autoreload 2

# Analyse the RBS data from Salis 2009

In [ ]:
import os

fn = os.path.join('data', 'RBSCalculatorData.xlsx')

In [ ]:
import pandas as pd

# Read the Excel file
excel_file = pd.ExcelFile(fn)

# Read the Salis 2009 sheet
df = excel_file.parse('Salis 2009')

# Display the column names
print("Column names:")
print(df.columns.tolist())
print("\nShape of dataframe:", df.shape)

In [ ]:
# Let's look at the first 20 rows to understand the structure
print("First 20 rows:")
for i in range(20):
    print(f"Row {i}: {df.iloc[i].tolist()}")

In [ ]:
# Read the Excel file again
import pandas as pd
import numpy as np

# Read the Salis 2009 sheet
df = pd.read_excel(fn, sheet_name='Salis 2009', header=None)

# Initialize lists to store the data
data_rows = []

# Process the spacer series section (rows 2-14)
for i in range(2, 15):
    row = df.iloc[i]
    if pd.notna(row[1]):  # Check if mRNA sequence exists
        data_rows.append({
            'mRNA sequence': row[1],
            'Predicted dG_tot [kcal/mol]': row[5],
            'Average Fluorescence [au]': row[2],
            'Standard Deviation of Fluorescence [au]': row[3],
            'n': 5,  # n=5 for this section
            'Doubling Time [hours]': np.nan,  # Not available for this section
            'Prediction #': np.nan,  # Not available for this section
            'Log Average Fluorescence [au]': row[7],
            'Spacer Series #': row[6]
        })

print(f"Processed {len(data_rows)} rows from spacer series section")

In [ ]:
# Continue processing the reverse engineered RBS series (rows 17-24)
for i in range(17, 25):
    row = df.iloc[i]
    if pd.notna(row[1]) and row[1] != 'mRNA sequence (containing the RBS and the beginning of the mRFP1 protein coding sequence, XbaI and SacI sites bolded)':
        data_rows.append({
            'mRNA sequence': row[1],
            'Predicted dG_tot [kcal/mol]': row[2],
            'Average Fluorescence [au]': row[3],
            'Standard Deviation of Fluorescence [au]': row[4],
            'n': 6,  # n=6 for this section
            'Doubling Time [hours]': row[5],
            'Prediction #': row[7],
            'Log Average Fluorescence [au]': row[8],
            'Spacer Series #': np.nan  # Not available for this section
        })

print(f"Total rows after reverse engineered section: {len(data_rows)}")

In [ ]:
# Continue processing the rest of the data
# Find where the next sections start
for i in range(25, len(df)):
    row = df.iloc[i]
    if pd.notna(row[1]) and isinstance(row[1], str) and len(row[1]) > 50:  # Likely an mRNA sequence
        # Check which format this row follows
        if pd.notna(row[2]) and pd.notna(row[3]) and pd.notna(row[4]):
            # This appears to be the format with Predicted dG_tot in column 2
            data_rows.append({
                'mRNA sequence': row[1],
                'Predicted dG_tot [kcal/mol]': row[2],
                'Average Fluorescence [au]': row[3],
                'Standard Deviation of Fluorescence [au]': row[4],
                'n': 6 if pd.notna(row[5]) else np.nan,
                'Doubling Time [hours]': row[5] if pd.notna(row[5]) else np.nan,
                'Prediction #': row[7] if pd.notna(row[7]) else np.nan,
                'Log Average Fluorescence [au]': row[8] if pd.notna(row[8]) else np.nan,
                'Spacer Series #': np.nan
            })

print(f"Total rows processed: {len(data_rows)}")

In [ ]:
# Remove the header row that was accidentally included
data_rows = [row for row in data_rows if not row['mRNA sequence'].startswith('mRNA sequence')]

# Create a DataFrame from the collected data
result_df = pd.DataFrame(data_rows)

# Reorder columns as requested
columns_order = [
    'mRNA sequence',
    'Predicted dG_tot [kcal/mol]',
    'Average Fluorescence [au]',
    'Standard Deviation of Fluorescence [au]',
    'n',
    'Doubling Time [hours]',
    'Prediction #',
    'Log Average Fluorescence [au]',
    'Spacer Series #'
]

result_df = result_df[columns_order]

print(f"Final dataframe shape: {result_df.shape}")
print("\nFirst few rows:")
print(result_df.head())

In [ ]:
# Now save this to a new Excel file with the extracted data
with pd.ExcelWriter('RBSCalculatorData_Extracted.xlsx', engine='openpyxl') as writer:
    result_df.to_excel(writer, sheet_name='Salis 2009 Extracted', index=False)

print("New Excel file created: RBSCalculatorData_Extracted.xlsx")
print(f"\nTotal number of rows extracted: {len(result_df)}")
print("\nColumn summary:")
for col in columns_order:
    non_null = result_df[col].notna().sum()
    print(f"  {col}: {non_null} non-null values")

In [ ]:
# Show sample rows from different sections
print("Sample rows from the extracted data:\n")

print("First 3 rows (Spacer Series):")
print(result_df.head(3).to_string())

print("\n\nRows 13-16 (Reverse engineered RBS series):")
print(result_df.iloc[13:16].to_string())

print("\n\nRows 100-103 (Later sections):")
print(result_df.iloc[100:103].to_string())